## Upload all of the initially parsed data to huggingface

In [7]:
import json
from datetime import datetime

def normalize_mongo_json(data):
    cleaned = []
    for item in data:
        obj = {}

        for k, v in item.items():
            if k in ["_id", "__v", "created_at"]:
                continue  # drop fields

            # unwrap MongoDB extended JSON types
            if isinstance(v, dict) and "$date" in v:
                # ensure it's ISO 8601 without Z
                dt = v["$date"]
                # remove Z if present
                dt = dt.rstrip("Z")
                obj[k] = dt
            else:
                obj[k] = v

        cleaned.append(obj)

    return cleaned


# Example usage:
raw_json = json.loads(open("prod.posts-mains.json").read())
clean_json = normalize_mongo_json(raw_json)

with open("output.json", "w") as f:
    json.dump(clean_json, f, indent=2)


In [15]:
import json
from datasets import load_dataset, Dataset, DatasetDict
dataset = Dataset.from_json("output.json")

dataset = dataset.map(
    lambda x: {
        "relevant": False,   # default boolean
        "sentiment": 0       # default integer (-1, 0, 1 allowed)
    }
)

Map:   0%|          | 0/247820 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict()
dataset_dict["source"] = dataset
dataset_dict.push_to_hub("tianharjuno/twitter-parse")

DatasetDict({
    source: Dataset({
        features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
        num_rows: 247820
    })
})

In [1]:
import json
from datasets import Dataset
import pandas as pd

OUT_DIR ="OUT/"
ASSET_DIR="assets/"
with open(ASSET_DIR + "dump-formatted.json", "r", encoding="utf-8") as file:
  RAW_DOCUMENTS = json.load(file)
dataframe = pd.DataFrame(RAW_DOCUMENTS)
dataframe = dataframe.drop(columns=["_id", "__v", "created_at"])
dataset = Dataset.from_pandas(dataframe)
dataset.push_to_hub("tianharjuno/twitter-parse", commit_description="Initial commit. not cleaned yet")


FileNotFoundError: [Errno 2] No such file or directory: 'assets/dump-formatted.json'

In [1]:
from datasets import load_dataset
ds = load_dataset("tianharjuno/twitter-parse", cache_dir="cache/")
raw_ds = ds["train"]

In [26]:
import re, unicodedata, jaconv, emoji

_URL      = re.compile(r'https?://\S+')
_MENTION  = re.compile(r'@\w+')
_REPEAT   = re.compile(r'(.)\1{2,}')       # ≥3 of same char
_WS       = re.compile(r'\s+')
_KUTI_CUT = re.compile(r'(?i)kutipan.*$', re.DOTALL)

# --- (MODIFIED) ---
# Catches "word" + "dari" + "domain.com" -> replaces with "word"
# Changed \w+ to \S+ to include punctuation like '!'
_DARI_URL_ATTACHED = re.compile(r'(\S+)dari\s+([a-z0-9.-]+\.[a-z]{2,})\b', re.I) 

# Catches " dari " + "domain.com" -> replaces with empty string
_DARI_URL_SPACED = re.compile(r'\s+dari\s+([a-z0-9.-]+\.[a-z]{2,})\b', re.I)

# Catches any standalone word that looks like a domain.com or domain.id
_DOMAIN_ONLY = re.compile(r'\b[a-z0-9.-]+\.[a-z]{2,}\b', re.I)

# --- (NEW) ---
# Catches any word ending in "dari" (e.g., "anarko!dari", "negaradari")
_DARI_STUCK = re.compile(r'(\S+)dari\b', re.I)

def cleantext(row: str):
    text = row["content"] #type: ignore
    text = unicodedata.normalize('NFKC', text)
    text = jaconv.z2h(text, kana=False, digit=True, ascii=True)
    text = text.replace("tanya grok", " ")
    text = text.replace("grokproductivitypasang", " ")
    text = text.replace('\\n', ' ').replace('\\r', ' ')
    
    # Handle standard URLs first
    text = _URL.sub(' <url> ', text)
    text = text.replace('<url> ini tidak tersedia', ' ')
    
    # --- (TYPO FIXED) ---
    text = _MENTION.sub('@USER', text)
    text = re.sub(r'^rt\s+', '', text, flags=re.I)
    text = re.sub(r'(\b\d{4})(?=[a-zA-Z])', r'\1 ', text)
    text = _KUTI_CUT.sub('', text)

    # --- RULE ORDER IS IMPORTANT ---
    # 1. Fix "word-dari domain.com" first
    text = _DARI_URL_ATTACHED.sub(r'\1', text) 
    # 2. Fix " word dari domain.com"
    text = _DARI_URL_SPACED.sub('', text)
    
    # --- (ADD THIS LINE) ---
    # 3. Fix any remaining "word-dari"
    text = _DARI_STUCK.sub(r'\1', text)
    # --- END OF ADDITION ---
    
    # 4. Clean up any other standalone domains
    text = _DOMAIN_ONLY.sub(' ', text)

    text = emoji.demojize(text, delimiters=(' ', ' '))
    text = _REPEAT.sub(r'\1', text)
    text = _WS.sub(' ', text).strip().lower()
    row["content"] = text #type: ignore
    return row

In [27]:
raw_ds = raw_ds.map(cleantext)

Map:   0%|          | 0/201583 [00:00<?, ? examples/s]

In [28]:
content_list = raw_ds["content"]

# Correct syntax: "\n".join(content_list)
# Added encoding="utf-8" to prevent errors with special characters
with open("reader.txt", "w", encoding="utf-8") as file:
    file.write("\n".join(content_list))

In [30]:
import pandas as pd

kamus_alay_read = pd.read_csv("kamus-alay/colloquial-indonesian-lexicon.csv")
kamus_alay_read = kamus_alay_read.drop(columns=["In-dictionary", "context", "category1", "category2", "category3"], errors="ignore")
alay_dict = dict(zip(kamus_alay_read["slang"], kamus_alay_read["formal"]))

def formalize(example):
    text = example["content"]
    words = text.split()
    normalized = []
    for w in words:
        normalized.append(alay_dict.get(w.lower(), w))
    example["content"] = " ".join(normalized)
    return example
raw_ds = raw_ds.map(formalize)

Map:   0%|          | 0/201583 [00:00<?, ? examples/s]

In [31]:
content_list = raw_ds["content"]

# Correct syntax: "\n".join(content_list)
# Added encoding="utf-8" to prevent errors with special characters
with open("reader.txt", "w", encoding="utf-8") as file:
    file.write("\n".join(content_list))

In [34]:
ds["cleaned"] = raw_ds

In [39]:
ds.push_to_hub("tianharjuno/twitter-parse", commit_description="cleaned up text")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/202 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Upload 0 LFS files: 0it [00:00, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Upload 0 LFS files: 0it [00:00, ?it/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tianharjuno/twitter-parse/commit/4b87c6da9e3d65295df90a581b960d98a8b01374', commit_message='Upload dataset', commit_description='cleaned up text', oid='4b87c6da9e3d65295df90a581b960d98a8b01374', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tianharjuno/twitter-parse', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tianharjuno/twitter-parse'), pr_revision=None, pr_num=None)

In [40]:
train_ds = ds["cleaned"]
test_ds = train_ds.shuffle(150).select(range(2000))

In [41]:
ds["test"] = test_ds
ds.push_to_hub("tianharjuno/twitter-parse", commit_description="Created a test dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/202 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Upload 0 LFS files: 0it [00:00, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tianharjuno/twitter-parse/commit/d3b2f6eb9178568c55179864899218f5eaf0365a', commit_message='Upload dataset', commit_description='Created a test dataset', oid='d3b2f6eb9178568c55179864899218f5eaf0365a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tianharjuno/twitter-parse', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tianharjuno/twitter-parse'), pr_revision=None, pr_num=None)